In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import os
import random
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, callbacks
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score

# -----------------------------
# CONFIG
# -----------------------------
DATASET_DIR = "/kaggle/input/chest-xray-pneumonia/chest_xray"
IMG_SIZE = 128  # Smaller size for lightweight model
BATCH_SIZE = 16
EPOCHS = 10
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)

# -----------------------------
# DATA PREPARATION
# -----------------------------
def collect_image_paths(dataset_root):
    import glob
    train_path = os.path.join(dataset_root, "train")
    test_path  = os.path.join(dataset_root, "test")

    def make_df(path):
        normal = glob.glob(os.path.join(path, "NORMAL", "*"))
        pneumonia = glob.glob(os.path.join(path, "PNEUMONIA", "*"))
        return pd.DataFrame({
            "image": normal + pneumonia,
            "class": ["Normal"] * len(normal) + ["Pneumonia"] * len(pneumonia)
        })

    return make_df(train_path), make_df(test_path)

def build_generator(df, img_size, batch_size, shuffle=True, augment=False):
    datagen = ImageDataGenerator(
        rescale=1./255,
        zoom_range=0.1 if augment else 0.0,
        width_shift_range=0.1 if augment else 0.0,
        height_shift_range=0.1 if augment else 0.0,
        horizontal_flip=augment
    )
    return datagen.flow_from_dataframe(
        df, x_col="image", y_col="class",
        target_size=(img_size, img_size),
        class_mode="binary",
        batch_size=batch_size, shuffle=shuffle
    )



2025-11-20 09:25:46.646781: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763630746.860133      39 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763630746.920912      39 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
# -----------------------------
# LOAD DATA
# -----------------------------
df_train_all, df_test = collect_image_paths(DATASET_DIR)
train_df, val_df = train_test_split(df_train_all, test_size=0.2,
                                    stratify=df_train_all["class"], random_state=SEED)

train_gen = build_generator(train_df, IMG_SIZE, BATCH_SIZE, augment=True)
val_gen   = build_generator(val_df, IMG_SIZE, BATCH_SIZE)
test_gen  = build_generator(df_test, IMG_SIZE, 1, shuffle=False)



Found 4172 validated image filenames belonging to 2 classes.
Found 1044 validated image filenames belonging to 2 classes.
Found 624 validated image filenames belonging to 2 classes.


In [3]:
# -----------------------------
# MODEL (MobileViT Hybrid - TFLite Compatible)
# -----------------------------

class TFLitePatchExtraction(layers.Layer):
    def __init__(self, patch_size):
        super().__init__()
        self.patch_size = patch_size
        
    def call(self, images):
        batch_size = tf.shape(images)[0]
        height = tf.shape(images)[1]
        width = tf.shape(images)[2]
        channels = tf.shape(images)[3]
        
        # Calculate output dimensions
        out_height = height // self.patch_size
        out_width = width // self.patch_size
        
        # Reshape to extract patches using depth_to_space approach
        x = tf.reshape(images, [batch_size, out_height, self.patch_size, out_width, self.patch_size, channels])
        x = tf.transpose(x, [0, 1, 3, 2, 4, 5])
        patches = tf.reshape(x, [batch_size, out_height * out_width, self.patch_size * self.patch_size * channels])
        
        return patches

class PositionEmbedding(layers.Layer):
    def __init__(self, num_patches, embed_dim):
        super().__init__()
        self.num_patches = num_patches
        self.embed_dim = embed_dim
        self.position_embedding = layers.Embedding(input_dim=num_patches, output_dim=embed_dim)

    def call(self, inputs):
        batch_size = tf.shape(inputs)[0]
        positions = tf.range(start=0, limit=self.num_patches, delta=1)
        
        # Get position embeddings and expand to batch size
        pos_emb = self.position_embedding(positions)
        pos_emb = tf.expand_dims(pos_emb, axis=0)
        pos_emb = tf.tile(pos_emb, [batch_size, 1, 1])
        
        return inputs + pos_emb

    def get_config(self):
        config = super().get_config()
        config.update({
            "num_patches": self.num_patches,
            "embed_dim": self.embed_dim
        })
        return config

def create_tflite_compatible_vit(img_size=128):
    """Create TFLite-compatible MobileViT hybrid"""
    
    inputs = keras.Input(shape=(img_size, img_size, 3))
    
    # CNN backbone for feature extraction
    x = layers.Conv2D(16, 3, strides=2, padding='same')(inputs)  # 64x64
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    
    x = layers.Conv2D(32, 3, strides=2, padding='same')(x)  # 32x32
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    
    x = layers.Conv2D(64, 3, strides=2, padding='same')(x)  # 16x16
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    
    # Extract patches using TFLite-compatible method
    patch_size = 4
    patches = TFLitePatchExtraction(patch_size=patch_size)(x)
    
    # Calculate number of patches
    num_patches = (16 // patch_size) ** 2  # 16x16 feature map divided by patch_size
    
    # Project patches and add position embedding
    projection = layers.Dense(48)(patches)
    encoded = PositionEmbedding(num_patches=num_patches, embed_dim=48)(projection)
    
    # Single transformer layer
    attn_output = layers.MultiHeadAttention(num_heads=2, key_dim=24)(encoded, encoded)
    transformed = layers.LayerNormalization(epsilon=1e-6)(attn_output + encoded)
    
    # FFN
    ffn = layers.Dense(96, activation='relu')(transformed)
    ffn = layers.Dropout(0.1)(ffn)
    ffn = layers.Dense(48)(ffn)
    transformed = layers.LayerNormalization(epsilon=1e-6)(ffn + transformed)
    
    # Global pooling and classification
    x = layers.GlobalAveragePooling1D()(transformed)
    x = layers.Dropout(0.3)(x)
    outputs = layers.Dense(1, activation='sigmoid')(x)
    
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

# Create the TFLite-compatible model
model = create_tflite_compatible_vit(img_size=IMG_SIZE)
model.compile(optimizer=keras.optimizers.Adam(1e-4),
              loss='binary_crossentropy',
              metrics=['binary_accuracy'])
model.summary()

print(f"✅ TFLite-compatible MobileViT created with {model.count_params():,} parameters")

2025-11-20 09:26:17.737778: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 128, 128,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 64, 64,    │        448 │ input_layer[0][0] │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 64, 64,    │         64 │ conv2d[0][0]      │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 64, 64,    │          0 │ batch_normalizat… │
│ (Activation)        │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 32, 32,    │      4,640 │ activation[0][0]  │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 32, 32,    │        128 │ conv2d_1[0][0]    │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_1        │ (None, 32, 32,    │          0 │ batch_normalizat… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 16, 16,    │     18,496 │ activation_1[0][… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 16, 16,    │        256 │ conv2d_2[0][0]    │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_2        │ (None, 16, 16,    │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ tf_lite_patch_extr… │ (None, 16, 1024)  │          0 │ activation_2[0][… │
│ (TFLitePatchExtrac… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 16, 48)    │     49,200 │ tf_lite_patch_ex… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ position_embedding  │ (None, 16, 48)    │        768 │ dense[0][0]       │
│ (PositionEmbedding) │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 16, 48)    │      9,408 │ position_embeddi… │
│ (MultiHeadAttentio… │                   │            │ position_embeddi… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 16, 48)    │          0 │ multi_head_atten… │
│                     │                   │            │ position_embeddi… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalization │ (None, 16, 48)    │         96 │ add[0][0]         │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 16, 96)    │      4,704 │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 93,009 (363.32 KB)

 Trainable params: 92,785 (362.44 KB)

 Non-trainable params: 224 (896.00 B)

✅ TFLite-compatible MobileViT created with 93,009 parameters


In [20]:
# -----------------------------
# TRAINING
# -----------------------------
es = callbacks.EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)
rlrop = callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.2, patience=2)

history = model.fit(train_gen,
                    validation_data=val_gen,
                    epochs=EPOCHS,
                    callbacks=[es, rlrop],
                    verbose=1)

model.save("pneumonia_mobilevit_hybrid.h5")
print("✅ Model saved as pneumonia_mobilevit_hybrid.h5")


Epoch 1/10
261/261 ━━━━━━━━━━━━━━━━━━━━ 79s 264ms/step - binary_accuracy: 0.7745 - loss: 0.4868 - val_binary_accuracy: 0.7433 - val_loss: 1.1979 - learning_rate: 1.0000e-04
Epoch 2/10
261/261 ━━━━━━━━━━━━━━━━━━━━ 67s 258ms/step - binary_accuracy: 0.8519 - loss: 0.3270 - val_binary_accuracy: 0.7462 - val_loss: 0.6004 - learning_rate: 1.0000e-04
Epoch 3/10
261/261 ━━━━━━━━━━━━━━━━━━━━ 68s 262ms/step - binary_accuracy: 0.8656 - loss: 0.2990 - val_binary_accuracy: 0.8592 - val_loss: 0.3565 - learning_rate: 1.0000e-04
Epoch 4/10
261/261 ━━━━━━━━━━━━━━━━━━━━ 68s 260ms/step - binary_accuracy: 0.8920 - loss: 0.2487 - val_binary_accuracy: 0.9186 - val_loss: 0.2096 - learning_rate: 1.0000e-04
Epoch 5/10
261/261 ━━━━━━━━━━━━━━━━━━━━ 69s 265ms/step - binary_accuracy: 0.9044 - loss: 0.2268 - val_binary_accuracy: 0.9234 - val_loss: 0.1871 - learning_rate: 1.0000e-04
Epoch 6/10
261/261 ━━━━━━━━━━━━━━━━━━━━ 69s 266ms/step - binary_accuracy: 0.9043 - loss: 0.2372 - val_binary_accuracy: 0.8640 - val_los

In [4]:
# -----------------------------
# TRAINING
# -----------------------------
es = callbacks.EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)
rlrop = callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.2, patience=2)

history = model.fit(train_gen,
                    validation_data=val_gen,
                    epochs=EPOCHS,
                    callbacks=[es, rlrop],
                    verbose=1)

model.save("pneumonia_mobilevit_hybrid.h5")
print("✅ Model saved as pneumonia_mobilevit_hybrid.h5")


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
261/261 ━━━━━━━━━━━━━━━━━━━━ 120s 421ms/step - binary_accuracy: 0.7470 - loss: 0.4985 - val_binary_accuracy: 0.7433 - val_loss: 1.5410 - learning_rate: 1.0000e-04
Epoch 2/10
261/261 ━━━━━━━━━━━━━━━━━━━━ 70s 269ms/step - binary_accuracy: 0.8713 - loss: 0.3015 - val_binary_accuracy: 0.8429 - val_loss: 0.3451 - learning_rate: 1.0000e-04
Epoch 3/10
261/261 ━━━━━━━━━━━━━━━━━━━━ 73s 278ms/step - binary_accuracy: 0.8898 - loss: 0.2646 - val_binary_accuracy: 0.4109 - val_loss: 1.5877 - learning_rate: 1.0000e-04
Epoch 4/10
261/261 ━━━━━━━━━━━━━━━━━━━━ 74s 281ms/step - binary_accuracy: 0.8981 - loss: 0.2438 - val_binary_accuracy: 0.9291 - val_loss: 0.1887 - learning_rate: 1.0000e-04
Epoch 5/10
261/261 ━━━━━━━━━━━━━━━━━━━━ 73s 279ms/step - binary_accuracy: 0.9076 - loss: 0.2141 - val_binary_accuracy: 0.7232 - val_loss: 0.6697 - learning_rate: 1.0000e-04
Epoch 6/10
261/261 ━━━━━━━━━━━━━━━━━━━━ 72s 278ms/step - binary_accuracy: 0.9181 - loss: 0.1820 - val_binary_accuracy: 0.3822 - val_lo

In [16]:
# import numpy as np

# # Smooth curves (optional)
# def smooth_curve(points, factor=0.8):
#     smoothed = []
#     last = points[0]
#     for p in points:
#         smoothed_val = last * factor + p * (1 - factor)
#         smoothed.append(smoothed_val)
#         last = smoothed_val
#     return smoothed

# # Accuracy Curve
# plt.figure(figsize=(9,6))
# plt.plot(smooth_curve(history.history['binary_accuracy']))
# plt.plot(smooth_curve(history.history['val_binary_accuracy']))
# plt.xlabel("Epochs")
# plt.ylabel("Accuracy")
# plt.legend(["Train Accuracy", "Validation Accuracy"])
# plt.title("Training vs Validation Accuracy")
# plt.grid(True)
# plt.show()

# # Loss Curve
# plt.figure(figsize=(9,6))
# plt.plot(smooth_curve(history.history['loss']))
# plt.plot(smooth_curve(history.history['val_loss']))
# plt.xlabel("Epochs")
# plt.ylabel("Loss")
# plt.legend(["Train Loss", "Validation Loss"])
# plt.title("Training vs Validation Loss")
# plt.grid(True)
# plt.show()
import keras
from matplotlib import pyplot as plt

# Assuming 'history' is the history object returned from model.fit

plt.plot(history.history['binary_accuracy'])
plt.plot(history.history['val_binary_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.show()

NameError: name 'x_test' is not defined

In [21]:

# -----------------------------
# EVALUATION
# -----------------------------
preds = model.predict(test_gen, verbose=1)
pred_labels = (preds > 0.5).astype(int).reshape(-1)
y_true = df_test["class"].map({"Normal":0, "Pneumonia":1}).values

print("Test Accuracy:", accuracy_score(y_true, pred_labels))
print(classification_report(y_true, pred_labels, target_names=["Normal","Pneumonia"]))
print("ROC-AUC:", roc_auc_score(y_true, preds))



624/624 ━━━━━━━━━━━━━━━━━━━━ 7s 10ms/step
Test Accuracy: 0.8141025641025641
              precision    recall  f1-score   support

      Normal       0.95      0.53      0.68       234
   Pneumonia       0.78      0.98      0.87       390

    accuracy                           0.81       624
   macro avg       0.86      0.76      0.78       624
weighted avg       0.84      0.81      0.80       624

ROC-AUC: 0.9216852947622179


In [22]:
# -----------------------------
# TFLITE INT8 QUANTIZATION
# -----------------------------
TFLITE_MODEL_PATH = "pneumonia_mobilevit_hybrid_int8.tflite"
N_REPRESENTATIVE = 100

def representative_dataset_gen_from_folder(img_folder, img_size, n=N_REPRESENTATIVE):
    input_name = model.inputs[0].name
    images = [os.path.join(img_folder, f) for f in os.listdir(img_folder) 
              if f.lower().endswith((".png", ".jpg", ".jpeg"))]
    images = random.sample(images, min(n, len(images)))
    
    for img_path in images:
        img = cv2.imread(img_path)
        if img is None:
            continue
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (img_size, img_size))
        img = np.expand_dims(img, axis=0).astype(np.uint8)  # INT8 input
        yield {input_name: img}

converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = lambda: representative_dataset_gen_from_folder(DATASET_DIR, IMG_SIZE)
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8

tflite_model = converter.convert()

with open(TFLITE_MODEL_PATH, "wb") as f:
    f.write(tflite_model)

print(f"✅ Fully INT8 quantized MobileViT Hybrid model saved as {TFLITE_MODEL_PATH}")

# Print model size information
original_size = os.path.getsize("pneumonia_mobilevit_hybrid.h5") / (1024 * 1024)
quantized_size = os.path.getsize(TFLITE_MODEL_PATH) / (1024 * 1024)

print(f"\n📊 Model Size Comparison:")
print(f"   Original H5: {original_size:.2f} MB")
print(f"   Quantized TFLite: {quantized_size:.2f} MB")
print(f"   Compression: {original_size/quantized_size:.1f}x smaller")

Saved artifact at '/tmp/tmp_heam6wv'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 128, 128, 3), dtype=tf.float32, name='keras_tensor_72')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  138793364536208: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138793364528720: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138793364527760: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138793364527952: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138793364526992: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138793364528144: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138793059516880: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138793059518416: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138793059518608: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138793059517840: TensorSpec(shape=(), dtype=tf.resource, name=None)
  13879305951

/usr/local/lib/python3.11/dist-packages/tensorflow/lite/python/convert.py:997: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(


✅ Fully INT8 quantized MobileViT Hybrid model saved as pneumonia_mobilevit_hybrid_int8.tflite

📊 Model Size Comparison:
   Original H5: 1.18 MB
   Quantized TFLite: 0.37 MB
   Compression: 3.2x smaller


W0000 00:00:1760375814.821026      37 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1760375814.821080      37 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.
fully_quantize: 0, inference_type: 6, input_inference_type: UINT8, output_inference_type: UINT8


In [3]:
import tensorflow as tf
import keras_cv  # if using MobileViT

# Load model (all custom layers available in this environment)
model_path = "/kaggle/input/pneumonia_mobilevit_hybrid/pneumonia_mobilevit_hybrid.h5"
model = tf.keras.models.load_model(model_path, compile=False)

# Convert to TFLite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

# Save TFLite file
tflite_path = "/kaggle/working/pneumonia_mobilevit_pi.tflite"
with open(tflite_path, "wb") as f:
    f.write(tflite_model)

# Confirm
print(f"✅ Conversion finished! TFLite file saved to: {tflite_path}")


FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = '/kaggle/input/pneumonia_mobilevit_hybrid/pneumonia_mobilevit_hybrid.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)